In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('processed_dataframe4.csv')

C:\Users\asche\AppData\Local\Temp\ipykernel_13084\1910874760.py:1: DtypeWarning: Columns (19,20,21,22,23,24,25,27,32,33,34,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_dataframe4.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199998 entries, 0 to 199997
Data columns (total 41 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   loan_type                                 199998 non-null  int64  
 1   loan_amount                               199998 non-null  int64  
 2   action_taken                              199998 non-null  int64  
 3   occupancy_type                            199998 non-null  int64  
 4   census_tract                              199929 non-null  float64
 5   applicant_ethnicity_1                     199238 non-null  float64
 6   co_applicant_ethnicity_1                  199717 non-null  float64
 7   applicant_race_1                          199743 non-null  float64
 8   applicant_race_2                          11514 non-null   float64
 9   co_applicant_race_1                       199904 non-null  float64
 10  co_applicant_race_2 

In [4]:
df.corr()

ValueError: could not convert string to float: '35-44'

In [ ]:
df_test = df.copy()

In [ ]:
columns_to_drop = ['reverse_mortgage', 'open_end_line_of_credit', 'total_points_and_fees', 'prepayment_penalty_term', 'negative_amortization', 'balloon_payment', 'other_nonamortizing_features', 'multifamily_affordable_units']
df_test = df_test.drop(columns=columns_to_drop)

In [ ]:
correlation_matrix = df_test.corr()

In [ ]:
correlation_matrix

In [ ]:
plt.matshow(correlation_matrix)
plt.show()

In [ ]:
correlation_with_target = correlation_matrix['action_taken'].drop('action_taken')

In [ ]:
correlation_with_target

In [ ]:
from itertools import cycle  
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])
bar_width = 0.5 

for i, (feature, correlation) in enumerate(correlation_with_target.items()):
    color = next(color_cycle)
    plt.bar(i, correlation, width=bar_width, color=color, label=feature)

# Add x ticks and labels
plt.xticks(range(len(correlation_with_target)), correlation_with_target.index)

plt.xlabel('Features')
plt.ylabel('Correlation with Target')
plt.title('Correlation with Target for Different Features')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Features')

plt.tight_layout()
plt.show()

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Check for missing values in each column
missing_values = df_test.isnull().sum()

# Display columns with missing values
columns_with_missing_values = missing_values[missing_values > 0]
print("Columns with missing values:")
print(columns_with_missing_values)

In [ ]:
target_variable = "action_taken"

In [ ]:
X = df_test.drop(columns=[target_variable])
y = df_test[target_variable]

In [ ]:
print(X.shape, y.shape)

In [ ]:
seed = 123

In [ ]:
# 1- loan originated
# 2- application approved but not accepted
# 3- application denied
# 4 - application withdrawn by applicant
# 5 - file closed for incompleteness
# 6 - purchased loan
# 7 - Pre-approval request denied
# 8 - pre-approval request approved but not accepted

y.value_counts()

In [ ]:
model = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs')
cv = RepeatedStratifiedKFold(n_splits = 6, n_repeats = 3, random_state=seed)
n_scores = cross_val_score(model, X, y, scoring = 'accuracy', cv=cv)

print('Mean accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

model.fit(X, y)

y_pred = model.predict(X)

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Create a StratifiedKFold object
skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=seed)

# Use it in cross_val_score
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=skf)

print('Mean accuracy: %.3f (%.3f)' % (n_scores.mean(), n_scores.std()))

# Note: You may need to handle the warning about the least populated class based on your specific dataset and context.


In [ ]:
accuracy = accuracy_score(y, y_pred)

print(classification_report(y, y_pred, zero_division=0))

In [ ]:
cm = confusion_matrix(y, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import joblib

# Save the model to a file
filename = 'logistic_regression_model.pkl'
joblib.dump(model, filename)
print('Model saved to', filename)